In [1]:
from enum import Enum
from typing import NamedTuple

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pytesseract
from pyaspeller import YandexSpeller
from spellchecker import SpellChecker

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.1
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.97
plt.rcParams['figure.subplot.bottom'] = 0.05
plt.rcParams['figure.subplot.hspace'] = 0.3
matplotlib.rc("image", cmap="gray_r")

# Prepare image
Загрузка, подготовка изображения для распознавания

In [3]:
# choose tool for show image

# Comment it if not use it
%matplotlib qt


class ShowTool(Enum):
    cv2= 'cv2'
    plt='plt'

SHOW_STYLE = ShowTool.plt

In [4]:
match SHOW_STYLE:
    case ShowTool.plt:

        def show(image: np.ndarray, y_size: int = 960):
            """
            Show image in cv2 window

            :param image: Image to show
            :param y_size: Window scaler, defaults to 960
            """
            plt.imshow(image)

    case ShowTool.cv2:

        def show(image: np.ndarray, y_size: int = 960):
            """
            Show image in cv2 window

            :param image: Image to show
            :param y_size: Window scaler, defaults to 960
            """

            dy, dx = image.shape[:2]
            size = (int(dx * y_size / dy), y_size)

            cv2.imshow('contours', cv2.resize(image, size))
            cv2.resizeWindow('contours', *size)
            cv2.waitKey()

## Load
Загружаем изображение и проверяем, правильное ли мы выбрали

In [314]:
img_real = cv2.imread(r"Data\2.jpg")
y, x, _ = img_real.shape
show(img_real)

## Align
Выравниваем (насколько возможно) изображение

In [271]:
def downscale_image(image: np.ndarray, max_size: int = 1920) -> np.ndarray:
    """
    Downscale image

    :param image: Input image
    :param max_size: Max size, defaults to 2048
    :return: Downsized image
    """

    scale = max_size / max(image.shape)
    return cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)


def make_square(image: np.ndarray) -> np.ndarray:
    """
    Make square from image to rotate it without a border cropping

    :param image: Supplemented image to square
    :return: Image
    """
    y, x = image.shape[:2]
    max_side = max(y, x)

    dy = max_side - y
    dx = max_side - x

    top = dy // 2
    bottom = dy - top
    left = dx // 2
    right = dx - left
    return cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])


def align(image: np.ndarray, tol: float = 5) -> np.ndarray:
    """
    Align image

    :param image: Image to align
    :param tol: Allowable angle deviation, defaults to 5
    :return: Aligned image
    """
    image_processed = cv2.Canny(image, 100, 200)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    image_processed = cv2.dilate(image_processed, kernel, iterations=2)

    non_zero_coordinates = cv2.findNonZero(image_processed)
    box = cv2.minAreaRect(non_zero_coordinates)
    angle = box[2]

    if (-tol < angle < tol) or (90 - tol < angle < 90 + tol) or (angle < -90 + tol):
        return image

    y, x = image.shape
    rotate_M = cv2.getRotationMatrix2D((x // 2, y // 2), angle, 1)
    return cv2.warpAffine(
        image.copy(),
        rotate_M,
        (int(x), int(y)),
        cv2.INTER_CUBIC,
        cv2.BORDER_REPLICATE,
    )

In [315]:
img_gray = cv2.cvtColor(img_real, cv2.COLOR_BGR2GRAY)
img_gray = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
img_gray = downscale_image(img_gray)

# Add little borders to emphasize black spaces on borders
border_add = 10
img_gray = cv2.copyMakeBorder(
    img_gray,
    border_add,
    border_add,
    border_add,
    border_add,
    cv2.BORDER_CONSTANT,
    value=[0, 0, 0],
)
img_gray = make_square(img_gray)
show(img_gray)

# Clasterization
Находим области с текстом о объединяем эти области в кластеры - абзацы для сохранения порядка слов при распознавания и исключения разрывов текста по середине абзаца.

Находим границы объектов, чтобы не включать области у краев, шум и тд. Размываем изображение гауссовским ядром и применяем расширение для образования связанных блоков текста, формируя маску для каждого блока текста. 

Главная проблема найти середину: соединить нужные абзацы - разделить большие блоки. Найти лучшее количесство расширений изображения можно по скорости уменьшения количества соединенных областей. Для этого несколько раз применяем `cv2.dilate` и находим минимум производной для значений `cv2.connectedComponents`<20 (чтобы отсечь начальное слияние маленьких квадратов в блоки), таким образом найдем точку перед которой произошло критическое уменьшение количества блоков - возможно слияние больших блоков между собой

In [316]:
def split(image: np.ndarray) -> dict[int, np.ndarray]:
    """
    Split image to connected blocks

    :param image: Image to split
    """
    bordered_image = cv2.Canny(image, 100, 200)
    blur_kernel = (9, 9)
    blur_image = cv2.GaussianBlur(bordered_image, blur_kernel, 2)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    quality = [100]
    dilated_image = blur_image.copy()
    while quality[-1] > 10:
        dilated_image = cv2.dilate(dilated_image, kernel)
        quality.append(cv2.connectedComponents(dilated_image)[0])

    start_quality = list(filter(lambda x: x < 20, quality))[0]
    start_i = quality.index(start_quality)

    quality_D = list(np.diff(quality[start_i:]))
    best_reduction_D = min(quality_D)
    best_iteration = start_i + quality_D.index(best_reduction_D) + 1

    dilated_image = cv2.dilate(blur_image, kernel, iterations=best_iteration)
    n_rectangles, segmented_mask = cv2.connectedComponents(dilated_image)
    cluster_list = [np.uint8(segmented_mask == i) * i for i in range(1, n_rectangles + 1)]
    return dict(
        [
            (np.max(mask), mask)
            for mask in sorted(cluster_list, key=lambda i: np.count_nonzero(i), reverse=True)
        ]
    )


def split_test(image: np.ndarray, max_n=30) -> dict[int, np.ndarray]:
    """
    Split image to connected blocks

    :param image: Image to split
    """
    bordered_image = cv2.Canny(image, 100, 200)
    blur_kernel = (9, 9)
    blur_image = cv2.GaussianBlur(bordered_image, blur_kernel, 2)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    quality = []
    dilated_image = blur_image.copy()
    for i in range(max_n):
        dilated_image = cv2.dilate(dilated_image, kernel)
        quality.append(cv2.connectedComponents(dilated_image)[0])
    return quality

In [317]:
clusters = split(img_gray)
show(sum(list(clusters.values())))

Вид функции количества областей от степени расширения:

In [311]:
res = np.array(split_test(img_gray))
plt.plot(res)
plt.twinx().plot(np.diff(res),c='r')

# Recognize
Распознаем текст на изображении и проверяем правильность написания

In [320]:
class RecognizeResult(NamedTuple):
    text: str
    # angle: int
    x: int
    y: int
    dx: int
    dy: int


speller = YandexSpeller(lang=['ru'])
text_blocks = []

test = []
for cluster in clusters.values():
    image_block = cv2.bitwise_and(img_gray, img_gray, mask=cluster)
    non_zero_coords = cv2.findNonZero(image_block)
    x, y, dx, dy = cv2.boundingRect(non_zero_coords)
    if dx * dy == 0 or np.count_nonzero(image_block) < 50:
        continue
    image_cropped = image_block[y : y + dy, x : x + dx]

    image_aligned = make_square(image_cropped)
    test.append(image_aligned)

    text = pytesseract.image_to_string(
        image_aligned,
        lang='rus',
        config='--psm 3',
    )
    text = speller.spelled(text)

    if len(text):
        text_blocks.append(RecognizeResult(text, x, y, dx, dy))

In [323]:
print(' '.join([i.text for i in sorted(text_blocks, key=lambda f: f.y)]))

ЕВРОГРУП
 Общество с ограниченной ответственностью
«Торговая Компания «ЕвроГруп»

''603053. с-Н.Новгород, ул, Борская, д.17, ИННЛКПП5256128400\525601901,
ОГРН114$256002451, ОКПО92320408, ОАО «НЕД-Банк» г. Н. Новгород, БИК042202705,
Расчетный счет40702810901050922597, Корр/счет 30101$10400000000705,

Е-тай; очговтирб2 @ай ту Телефон/факс: 8 ($31) 253.98-60.253-37-91
 ООО«СтрейЮрист»
 мая 2015 г.
 № 364 от « 2$»
 ОТЗЫВ
 В Стройюристе мы оформляли сертификат 15О 9001 для участия в
тендере. Нас сразу привлекла стоимость в 19 тысяч рублей. И срочность
оформления серткфиката, так как сроки у нас «горели». По телефону
получили грамотную консультацию по 150 и ская документа был у нас в
этот же день. В результате тендер мы выиграли и успешно работаем.
Спасибо Стройюрист, за качественную работу!
 С уважением,
Директор
СО0 «ТК Еврогруп»:

 Белов Е.А.

